# More advanced exercises

Try creating a 3-way, perhaps bringing Gemini into the conversation! One student has completed this - see the implementation in the community-contributions folder.

The most reliable way to do this involves thinking a bit differently about your prompts: just 1 system prompt and 1 user prompt each time, and in the user prompt list the full conversation so far.

In [ ]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display

from openai import OpenAI

In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

In [ ]:
anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"

In [9]:
openai = OpenAI()
anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)

In [10]:
gpt_model = "gpt-4.1-mini"
claude_model = "claude-3-5-haiku-latest"
gemini_model= "gemini-2.5-pro"

In [ ]:
conversation = "Blake: Hi" + "\n\nCharlie: Hey there" 
print(conversation)

import time
import random

# Define system prompts (these don't change)
system_alex = """
You are Alex, a chatbot who is very argumentative; you disagree with anything in the conversation and you challenge everything, in a snarky way.
You are in a conversation with Blake and Charlie.
"""

system_blake = """
You are Blake, a chatbot who is very shy but extremely smart; you try to deeply understand everything that has been that, and answer in a sharp, short and eloquent way.
You are in a conversation with Alex and Charlie.
"""

system_charlie = """
You are Charlie, a chatbot who is not very smart but has a certain air of shrewdness; you try to emphasize practical aspects of the conversation that could be useful in daily life.
You are in a conversation with Alex and Blake.
"""

def make_api_call_with_retry(client, model, messages, max_retries=3, name=""):
    """Make API call with exponential backoff retry logic"""
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(model=model, messages=messages)
            return response
        except Exception as e:
            if "503" in str(e) or "overloaded" in str(e).lower() or attempt < max_retries - 1:
                wait_time = (2 ** attempt) + random.uniform(0, 1)  # Exponential backoff with jitter
                print(f"  {name} API error (attempt {attempt + 1}/{max_retries}): {str(e)[:100]}")
                print(f"  Waiting {wait_time:.1f} seconds before retry...")
                time.sleep(wait_time)
            else:
                raise  # Re-raise if it's the last attempt and not a 503

for i in range(1, 8):
    print(f"\n--- Iteration {i} ---")
    
    # Regenerate user prompts with updated conversation INSIDE the loop
    user_alex = f"""
You are Alex, in conversation with Blake and Charlie.
The conversation so far is as follows:
{conversation}
Now with this, respond with what you would like to say next, as Alex.
"""
    
    messages_alex = [{"role": "system", "content": system_alex}, {"role":"user", "content": user_alex}]
    response_alex = make_api_call_with_retry(openai, gpt_model, messages_alex, name="Alex (OpenAI)")
    conversation += "\n\nAlex:" + response_alex.choices[0].message.content
    print("\n\nAlex:" + response_alex.choices[0].message.content)
    time.sleep(2.0)  # Increased to 2 seconds between calls

    # Regenerate user prompt for Blake
    user_blake = f"""
You are Blake, in conversation with Alex and Charlie.
The conversation so far is as follows:
{conversation}
Now with this, respond with what you would like to say next, as Blake.
"""
    
    messages_blake = [{"role": "system", "content": system_blake}, {"role":"user", "content": user_blake}]
    response_blake = make_api_call_with_retry(anthropic, claude_model, messages_blake, name="Blake (Anthropic)")
    conversation += "\n\nBlake:" + response_blake.choices[0].message.content
    print("\n\nBlake:" + response_blake.choices[0].message.content)
    time.sleep(2.0)  # Increased to 2 seconds

    # Regenerate user prompt for Charlie
    user_charlie = f"""
You are Charlie, in conversation with Alex and Blake.
The conversation so far is as follows:
{conversation}
Now with this, respond with what you would like to say next, as Charlie.
"""
    
    messages_charlie = [{"role": "system", "content": system_charlie}, {"role":"user", "content": user_charlie}]
    response_charlie = make_api_call_with_retry(gemini, gemini_model, messages_charlie, name="Charlie (Gemini)")
    conversation += "\n\nCharlie:" + response_charlie.choices[0].message.content
    print("\n\nCharlie:" + response_charlie.choices[0].message.content)
    time.sleep(2.0)  # Increased to 2 seconds